In [1]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [2]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [5]:
import csv
import json
import re
from typing import Dict, Set, Tuple

def estrai_schema_da_domanda(llm: AzureChatOpenAI, domanda: str) -> Dict:
    """
    Analizza una domanda utilizzando il modello LLM per estrarre entità e relazioni.
    """
    prompt = f"""
    Analizza la seguente domanda e determina:
    - Le entità principali (es. Piatto, Ristorante, Pianeta, Tecnica)
    - Le relazioni tra di esse (es. Piatto PREPARATO_CON Tecnica, Ristorante SERVE Piatto)
    
    Domanda: "{domanda}"
    
    Rispondi in JSON nel formato:
    {{
        "entità": ["..."],
        "relazioni": [{{"soggetto": "...", "relazione": "...", "oggetto": "..."}}]
    }}
    """

    response = llm.predict(prompt)

    # Debugging: Stampiamo la risposta grezza del modello
    print(f"DEBUG - Risposta LLM per la domanda:\n{domanda}\n{response}\n")

    # Se la risposta è vuota
    if not response or response.strip() == "":
        print("⚠️ ERRORE: Risposta vuota dal modello!")
        return {"entità": [], "relazioni": []}

    # 1. Rimuove blocchi Markdown tipo ```json ... ```
    match = re.search(r"```json\s*(.*?)\s*```", response, re.DOTALL)
    if match:
        response = match.group(1).strip()

    # 2. Rimuove caratteri speciali residui che potrebbero disturbare il parsing
    response = response.replace("\n", "").replace("\t", "").strip()

    # 3. Prova a decodificare il JSON
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        print(f"⚠️ ERRORE: Il modello ha restituito un JSON non valido. Errore: {e}")

        # Se fallisce, proviamo a correggerlo rimuovendo caratteri extra
        try:
            response = re.sub(r"[^\x20-\x7E]+", "", response)  # Rimuove caratteri non ASCII
            return json.loads(response)
        except json.JSONDecodeError:
            print("⚠️ ERRORE: Correzione fallita. Scartiamo questa risposta.")
            return {"entità": [], "relazioni": []}

def genera_schema_grafo(csv_path: str, llm: AzureChatOpenAI) -> Dict:
    """
    Legge un file CSV di domande, analizza le entità e relazioni per generare uno schema di grafo.
    """
    schema = {"entità": set(), "relazioni": set()}

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            domanda = row[0]
            risultato_json = estrai_schema_da_domanda(llm, domanda)
            
            schema["entità"].update(risultato_json["entità"])
            for relazione in risultato_json["relazioni"]:
                schema["relazioni"].add((relazione["soggetto"], relazione["relazione"], relazione["oggetto"]))

    return schema

# Esempio di utilizzo
csv_file = "C:/Users/lparigi/Downloads/letspizza/Hackapizza Dataset/domande.csv"
schema_grafo = genera_schema_grafo(csv_file, llm)

# Stampa dello schema estratto
print("Entità identificate:")
print(schema_grafo["entità"])

print("\nRelazioni identificate:")
for r in schema_grafo["relazioni"]:
    print(f"{r[0]} -[{r[1]}]-> {r[2]}")


DEBUG - Risposta LLM per la domanda:
domanda
Sembra che tu non abbia fornito una domanda specifica da analizzare. Per favore, inserisci la domanda che desideri analizzare.

⚠️ ERRORE: Il modello ha restituito un JSON non valido. Errore: Expecting value: line 1 column 1 (char 0)
⚠️ ERRORE: Correzione fallita. Scartiamo questa risposta.
DEBUG - Risposta LLM per la domanda:
Quali sono i piatti che includono le Chocobo Wings come ingrediente?
```json
{
    "entità": ["Piatti", "Chocobo Wings"],
    "relazioni": [
        {
            "soggetto": "Piatti",
            "relazione": "INCLUDONO",
            "oggetto": "Chocobo Wings"
        }
    ]
}
```

DEBUG - Risposta LLM per la domanda:
Quali piatti dovrei scegliere per un banchetto a tema magico che includa le celebri Cioccorane?
```json
{
    "entità": ["Piatto", "Banchetto", "Tema Magico", "Cioccorane"],
    "relazioni": [
        {"soggetto": "Piatto", "relazione": "SCEGLIERE_PER", "oggetto": "Banchetto"},
        {"soggetto": "Ban

KeyError: 'oggetto'